<center>
  <h1 style="color: #4a90e2;">Space Titanic</h1>
  <h2 style="color: #50e3c2;">🚀✨ 0.814 🔧『Optuna XGB』| Space Titanic</h2>
  <h3 style="color: #7f8c8d;">If you find my notebook helpful, support with one upvote, thank you</h3>
  <h4 style="color: #bdc3c7;">Created by Mr.Tao 10/11/2023 👍</h4>
</center>


# <span style="color: #2E8B57;">📁 Description</span>

## <span style="color: #4682B4;">📣  Recommended Competition</span>

We highly recommend Titanic - Machine Learning from Disaster to get familiar with the basics of machine learning and Kaggle competitions.

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

Help save them and change history!

### <span style="color: #6A5ACD;">Dataset Description</span>

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system

### <span style="color: #FF6347;">File and Data Field Descriptions</span>

**train.csv**  
Personal records for about two-thirds of the passengers, to be used as training data.

- **PassengerId**: A unique Id for each passenger. Each Id takes the form `gggg_pp` where `gggg` indicates a group the passenger is traveling with and `pp` is their number within the group. People in a group are often family members, but not always.

- **HomePlanet**: The planet the passenger departed from, typically their planet of permanent residence.

- **CryoSleep**: Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.

- **Cabin**: The cabin number where the passenger is staying. Takes the form `deck/num/side`, where `side` can be either `P` for Port or `S` for Starboard.

- **Destination**: The planet the passenger will be debarking to.

- **Age**: The age of the passenger.

- **VIP**: Whether the passenger has paid for special VIP service during the voyage.

- **RoomService**: Amount the passenger has billed at the Room Service.

- **FoodCourt**: Amount the passenger has billed at the Food Court.

- **ShoppingMall**: Amount the passenger has billed at the Shopping Mall.

- **Spa**: Amount the passenger has billed at the Spa.

- **VRDeck**: Amount the passenger has billed at the VR Deck.

- **Name**: The first and last names of the passenger.

- **Transported**: Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.

**test.csv**  
Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of `Transported` for the passengers in this set.

- **PassengerId**: Id for each passenger in the test set.

- **Transported**: The target. For each passenger, predict either `True` or `False`.

**sample_submission.csv**  
A submission file in the correct format.


<a id="1"></a>
# **<center><span style="color:#005dc4;">Import Libraries</span></center>**

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
import pandas as pd
import numpy as np
import base64
import seaborn as sns
import matplotlib.pyplot as plt
import os
import random
import gc

import matplotlib.pyplot as plotter
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
import optuna
import xgboost as xgb
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import datasets
optuna.logging.set_verbosity(optuna.logging.WARNING)
from lightgbm import *
pd.set_option("display.max_columns", None)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import eli5
from eli5.sklearn import PermutationImportance

import warnings
warnings.filterwarnings('ignore')

<a id="2"></a>
# **<center><span style="color:#005dc4;">Read Dataset</span></center>**

In [ ]:
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

<a id="3"></a>
# **<center><span style="color:#005dc4;">EDA</span></center>**

In [ ]:
print('train')
display(train.isnull().sum())
print('test')
display(test.isnull().sum())

In [ ]:
print('train')
display(train.info())
print('test')
display(test.info())

In [ ]:
plt.figure(figsize = (10, 2))

plt.subplot(1, 3, 1)
plt.title("Training Set")
sns.heatmap(train.isnull())

plt.subplot(1, 3, 3)
plt.title("Testing Set")
sns.heatmap(test.isnull())
plt.show()

In [ ]:
train

In [ ]:
sns.countplot(x = train["Transported"])
plotter.xticks(rotation = 0);

In [ ]:
X = train.drop(columns=['Transported'],axis=1)#drop
y = train["Transported"]

<a id="4"></a>
# **<center><span style="color:#005dc4;">Optuna</span></center>**

In [ ]:
#optuna

#def objective(trial):
#    params = {
#        'random_seed':123,
#        'n_estimators'    :trial.suggest_int('n_estimators', 300, 1000),
#        'num_leaves'      :trial.suggest_int('num_leaves', 4, 32),
#        'max_depth'       :trial.suggest_int("max_depth",1,10),
#        'lambda'          : trial.suggest_loguniform("lambda", 1e-8, 1.0),
#        'subsample'       : trial.suggest_float("subsample", 0.01, 1.0, step = 0.1),   
#        'reg_lambda'      : trial.suggest_loguniform("lambda", 1e-8, 1.0),
#        'reg_alpha'       : trial.suggest_loguniform("alpha", 1e-8, 1.0)},
#        'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0, step = 0.1)   
            
#    model = xgb.XGBClassifier(**params)
#    model.fit(X,y)
#    y_pred = model.predict(X)
#    score = accuracy_score(y, y_pred)
    
#    return score

In [ ]:
#study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=123))
#study.optimize(objective, n_trials=50) 
#study.best_params

In [ ]:
def get_score(model,X,y):
    n = cross_val_score(model,X,y,scoring ='accuracy',cv=10)
    return n

In [ ]:
params_XGB_best ={'lambda': 3.06, 
             'alpha': 4.582, 
             'colsample_bytree': 0.93, 
             'subsample': 0.96, 
             'n_estimators': 725, 
             'max_depth': 5, 
             'learning_rate': 0.05    }

In [ ]:
def t_fold(X,y,n_splits): 
    params= {'lambda': 3.06, 
             'alpha': 4.582, 
             'colsample_bytree': 0.93, 
             'subsample': 0.96, 
             'n_estimators': 7250, 
             'max_depth': 5, 
             'learning_rate': 0.05 ,
             'early_stopping_rounds':200,}
    results=[]
    n_iterations=[]
    skf = StratifiedKFold(n_splits=n_splits)
    for train_index, test_index in skf.split(X, y):
        train_X, valid_X = X.iloc[train_index], X.iloc[test_index]
        train_y, valid_y = y.iloc[train_index], y.iloc[test_index]
        model = xgb.XGBClassifier(**params).fit(train_X,train_y,
                                      eval_set=[(valid_X,valid_y)],
                                      verbose=0
                                     )  
        n_iteration = model.get_booster().best_iteration
        n_iterations.append(n_iteration)
        result = accuracy_score(valid_y,(model.predict(valid_X)))
        results.append(result)
        i=int(sum(n_iterations)/len(n_iterations))
    print("Average n_ite=" + str(i))
    print("% of scatter =" + str(np.std(n_iterations)/i))    
    n=sum(results)/len(results) 
    print (n)
    print("FIIINISH__________________________________\n")
    return n

<a id="5"></a>
# **<center><span style="color:#005dc4;">Preprocessing</span></center>**

In [ ]:
train_test=train._append(test,ignore_index=True)

In [ ]:
print(train_test.isnull().sum())
sns.heatmap(train_test.isnull())

# **<center><span style="color:#005dc4;">Expenses & CryoSleep</span></center>**

In [ ]:
Expenses_columns = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

In [ ]:
train_test.loc[:,Expenses_columns]=train_test.apply(lambda x: 0 if x.CryoSleep == True else x,axis =1)

# **<center><span style="color:#005dc4;">CryoSleep & Expenses</span></center>**

In [ ]:
train_test['Expenses'] = train_test.loc[:,Expenses_columns].sum(axis=1)

In [ ]:
train_test.loc[:,['CryoSleep']]=train_test.apply(lambda x: True if x.Expenses == 0 and pd.isna(x.CryoSleep) else x,axis =1)

# **<center><span style="color:#005dc4;">N/A data</span></center>**

In [ ]:
train_test.Name = train_test.Name.fillna('Unknown Unknown')

# **<center><span style="color:#005dc4;">Cabin process</span></center>**

In [ ]:
train_test.loc[:,['Room']] = train_test.PassengerId.apply(lambda x: x[0:4] )

In [ ]:
guide_VIP=train_test.loc[:,['Room','VIP']].dropna().drop_duplicates('Room')
guide_Cabin=train_test.loc[:,['Room','Cabin']].dropna().drop_duplicates('Room')
guide_HomePlanet=train_test.loc[:,['Room','HomePlanet']].dropna().drop_duplicates('Room')
guide_Destination=train_test.loc[:,['Room','Destination']].dropna().drop_duplicates('Room')
train_test=pd.merge(train_test,guide_Cabin,how="left",on='Room',suffixes=('','_y'))
train_test=pd.merge(train_test,guide_VIP,how="left",on='Room',suffixes=('','_y'))
train_test=pd.merge(train_test,guide_HomePlanet,how="left",on='Room',suffixes=('','_y'))
train_test=pd.merge(train_test,guide_Destination,how="left",on='Room',suffixes=('','_y'))

In [ ]:
train_test.loc[:,['VIP']]=train_test.apply(lambda x: x.VIP_y if pd.isna(x.VIP) else x,axis=1)
train_test.loc[:,['Cabin']]=train_test.apply(lambda x:  x.Cabin_y if pd.isna(x.Cabin) else x,axis=1)
train_test.loc[:,['HomePlanet']]=train_test.apply(lambda x:  x.HomePlanet_y if pd.isna(x.HomePlanet) else x,axis=1)
train_test.loc[:,['Destination']]=train_test.apply(lambda x:  x.Destination_y if pd.isna(x.Destination) else x,axis=1)

In [ ]:
train_test[train_test.Cabin.isnull()]

In [ ]:
fig,ax = plt.subplots(figsize=(30,10))
pd.set_option('display.max_rows', 100)
print(train_test.isnull().sum())
sns.heatmap(train_test.isnull())

# **<center><span style="color:#005dc4;">Age process</span></center>**

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
analys = train_test.loc[:,['Age','Expenses']]
ax.scatter(analys.Age,analys.Expenses)

In [ ]:
pd.set_option('display.max_rows', 100)
print(train_test.isnull().sum())
sns.heatmap(train_test.isnull())

# **<center><span style="color:#005dc4;">HomePlanet & Destination</span></center>**

In [ ]:
analys = train_test.loc[:,['HomePlanet','Destination']]
analys['numeric'] =1
analys.groupby(['Destination','HomePlanet']).count()

In [ ]:
pd.set_option('display.max_rows', 100)
print(train_test.isnull().sum())
sns.heatmap(train_test.isnull())

# **<center><span style="color:#005dc4;">Split Cabin column</span></center>**

In [ ]:
train_test.loc[:,['Cabin_1']] = train_test.Cabin.str.split("/",expand=True).iloc[:,0]
train_test.loc[:,['Cabin_2']] = train_test.Cabin.str.split("/",expand=True).iloc[:,1]
train_test.loc[:,['Cabin_3']] = train_test.Cabin.str.split("/",expand=True).iloc[:,2]
train_test

In [ ]:
train_test.loc[:,['FirstName']] = train_test.Name.str.split(" ",expand=True).iloc[:,0]
train_test.loc[:,['SecondName']] = train_test.Name.str.split(" ",expand=True).iloc[:,1]
train_test['Name_key']=train_test['SecondName']+train_test['Room']
train_test

# **<center><span style="color:#005dc4;">SimpleImputer + OneHotEncoder</span></center>**

In [ ]:
num_cols = ['ShoppingMall','FoodCourt','RoomService','Spa','VRDeck','Expenses','Age']
cat_cols = ['CryoSleep','Cabin_1','Cabin_3','VIP','HomePlanet','Destination']
transported=['Transported']
train_test = train_test[num_cols+cat_cols+transported].copy()

num_imp = SimpleImputer(strategy='mean')
cat_imp = SimpleImputer(strategy='most_frequent')
ohe = OneHotEncoder (handle_unknown='ignore',sparse = False)


train_test[num_cols] = pd.DataFrame(num_imp.fit_transform(train_test[num_cols]),columns=num_cols)
train_test[cat_cols] = pd.DataFrame(cat_imp.fit_transform(train_test[cat_cols]),columns=cat_cols)
temp_train = pd.DataFrame(ohe.fit_transform(train_test[cat_cols]),columns= ohe.get_feature_names_out())
train_test = train_test.drop(cat_cols,axis=1)
train_test = pd.concat([train_test,temp_train],axis=1)
train_test

# **<center><span style="color:#005dc4;">split train and test set</span></center>**

In [ ]:
train = train_test[train_test['Transported'].notnull()].copy()
train.Transported =train.Transported.astype('int')
test = train_test[train_test['Transported'].isnull()].drop("Transported",axis=1)
display(train)
display(test)

In [ ]:
X = train.drop('Transported',axis=1)
y = train.Transported

In [ ]:
X,y = shuffle(X,y)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

In [ ]:
print(get_score(xgb.XGBClassifier(**params_XGB_best),X,y).mean())

# **<center><span style="color:#005dc4;">Outliers</span></center>**

In [ ]:
features_isolation =['ShoppingMall','FoodCourt','RoomService','Spa','VRDeck','Age']

In [ ]:
from sklearn.ensemble import IsolationForest

isf = IsolationForest(n_jobs=-1, random_state=1,n_estimators=100,contamination=0.003)
isf.fit(X[features_isolation], y)

rows = pd.DataFrame(isf.predict(X[features_isolation]),columns=['feature'])
rows_ind = rows[rows.feature == 1]
results = pd.DataFrame()
results['results'] = - isf.score_samples(X[features_isolation])
results['chance'] = isf.decision_function(X[features_isolation])
results['yes/no'] = isf.predict(X[features_isolation])

new = results[results['yes/no']==1]
old= results[results['yes/no']==-1]
fig,ax = plt.subplots(figsize=(20,10))
ax.set_facecolor('black')
ax = plt.scatter(new.index,new.results,marker=".")
ax = plt.scatter(old.index,old.results,color='white',marker="*")


In [ ]:
X_1 = X.iloc[rows_ind.index].reset_index(drop=True)
y_1 = y.iloc[rows_ind.index].reset_index(drop=True) 

In [ ]:
print(get_score(xgb.XGBClassifier(**params_XGB_best),X_1,y_1).mean())

In [ ]:
drop_list=['ShoppingMall','Age','CryoSleep_True','HomePlanet_Earth','HomePlanet_Europa',
'VIP_True','HomePlanet_Mars','Destination_PSO J318.5-22','VIP_False',
'Destination_55 Cancri e','FoodCourt','Destination_TRAPPIST-1e']

In [ ]:
X=X.drop(drop_list,axis=1)
test=test.drop(drop_list,axis=1)

In [ ]:
print(get_score(xgb.XGBClassifier(**params_XGB_best),X,y).mean())

<a id="6"></a>
# **<center><span style="color:#005dc4;">KMeansSMOTE</span></center>**

In [ ]:
from imblearn.over_sampling import KMeansSMOTE
sm = KMeansSMOTE(sampling_strategy=1, n_jobs=-1)
X_sm, y_sm = sm.fit_resample(X, y)
X = X_sm
y = y_sm

In [ ]:
y.value_counts()

<a id="7"></a>
# **<center><span style="color:#005dc4;">Predict</span></center>**

In [ ]:
pred_XGB = (xgb.XGBClassifier(**params_XGB_best).fit(X,y)).predict(test)
pred_XGB

<a id="8"></a>
# **<center><span style="color:#005dc4;">Submission</span></center>**

In [ ]:
Submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
Submission

In [ ]:
Submission['Transported'] = pred_XGB
Submission

In [ ]:
#1/0 → TRUE/FALSE
Submission['Transported']=Submission['Transported']>0.5
Submission.to_csv('Submission_XGB.csv', index=False)
Submission

<div class="alert alert-block alert-info" style="font-size:20px; background-color: #E4FFFF; font-family:verdana; color: #000115; border: 1px #000115 solid">
 I'm a beginner in the python world, not in undergrad or for work,just interest, and I'm still learning a lot of different ways to do it. I hope you'll give me your vote as well as comments to help me improve in the future, thanks!    Mr.Tao